Nepal Earthquake OSM Data Exploration
=====================================

In [2]:
require '/opt/epic-osm/epic-osm.rb'
analysis_window = './nepal-earthquake.yml'
o  = EpicOSM.new(analysis_window: analysis_window)
aw = o.analysis_window

Successfully initialized Analysis Window: Nepal Earthquake



#<AnalysisWindow:0x007f3da7987ab0 @bounding_box=#<BoundingBox:0x007f3da798ce20 @bbox_array=["80.0134", "26.3033", "88.2202", "30.5149"], @bottom_left=[80.0134, 26.3033], @top_right=[88.2202, 30.5149], @active=false>, @time_frame=#<TimeFrame:0x007f3da798f4e0 @start_date=2015-04-25 00:00:00 +0000, @end_date=2015-05-31 00:00:00 +0000, @active=true>, @max_area=1000000000000, @min_area=1, @title="Nepal Earthquake", @changeset_tags=nil>

## Visualize some changesets information!

In [3]:
cSets_per_4hours = aw.changesets_x_hour(step: 4);
nil

{:unit=>:hour, :step=>4, :constraints=>{}, :collection=>"changesets", :type=>DomainObject::Changeset}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 00:00:00 +0000, "$lt"=>2015-04-25 04:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 04:00:00 +0000, "$lt"=>2015-04-25 08:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 08:00:00 +0000, "$lt"=>2015-04-25 12:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 12:00:00 +0000, "$lt"=>2015-04-25 16:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 16:00:00 +0000, "$lt"=>2015-04-25 20:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-25 20:00:00 +0000, "$lt"=>2015-04-26 00:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-26 00:00:00 +0000, "$lt"=>2015-04-26 04:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-26 04:00:00 +0000, "$lt"=>2015-04-26 08:00:00 +0000}}
Query complete for: {:created_at=>{"$gte"=>2015-04-26 08:0

In [161]:
users_per_4hours = cSets_per_4hours.collect{|bucket| 
  {time: bucket[:start_date],
   all_users: bucket[:objects].collect{|x| x.user}.uniq,
   hot_users: bucket[:objects].select{|c| c.comment =~ /hot/}.collect{|x| x.user}.uniq
  }
}

counts_per_4_hours = users_per_4hours.collect{ |d| 
  { time:      d[:time],
    all_users: d[:all_users].count,
    hot_users: d[:hot_users].count
  }
}; nil

In [162]:
puts "All users: #{ users_per_4hours.collect{|x| x[:all_users] }.flatten.uniq.length }"
puts "HOT users: #{ users_per_4hours.collect{|x| x[:hot_users] }.flatten.uniq.length }"
nil

All users: 7068
HOT users: 6075


In [169]:
#Write out JSON to visualize:
require 'json'
File.write('json/user_counts_per_4hour.json', counts_per_4_hours.to_json)

14586

In [165]:
require 'rubyvis'
w = 800
h = 600

#Scale appropriately
colors=pv.Colors.category20()

# x = pv.Scale.linear(0, counts_per_4_hours.length}).range(0, w)
x = pv.Scale.ordinal(counts_per_4_hours, lambda {|d| d.time}).split_banded(0,w,4/5.0)

y = pv.Scale.linear(0, counts_per_4_hours.collect{|x| x[:all_users]}.max).range(0, h)
k = x.range_band / 2

fill = pv.colors("lightblue", "darkgray")

format = pv.Format.date("%d")

vis = pv.Panel.new()
    .width(w)
    .height(h)
    .margin(19.5)
    .right(40);

panel = vis.add(pv.Panel)
    .data(counts_per_4_hours)
    .left(lambda {|d| x.scale(d.time)})
    .width(x.range_band);
    
panel.add(pv.Bar)
    .data(counts_per_4_hours)
    .bottom(0)
    .width(k)
    .left(lambda { self.index * k})
    .height(lambda {|t, d| y.scale(d.send(t))})
    .fill_style(lambda {|f| a=fill.scale(self.index); return a})
    .stroke_style(lambda {|d| fill_style ? fill_style.darker : pv.color('black')})
    .line_width(1);

panel.anchor("bottom").add(pv.Label)
.visible(lambda { (self.index % 3)==0})
.text_baseline("top")
    .text_margin(10)
    .text(lambda {|d| format.format(d.date)})
    
vis.add(pv.Rule)
    .data(y.ticks())
    .bottom(lambda {|d| y.scale(d)})
    .stroke_style(lambda {|i|  i!=0 ? pv.color("rgba(255, 255, 255, .5)") : pv.color("black")})
  .anchor("right").add(pv.Label)
  .visible(lambda { (self.index & 1)==0})
    .text_margin(6);

# #/* The bars. */
# colors=pv.Colors.category20()

# bar = vis.add(pv.Panel)
#     .data(data)
#     .top(lambda {y.scale(self.index)})
#     .height(y.range_band)
#   .add(pv.Bar)
#     .data(lambda {|d| d})
#     .top(lambda {self.index * y.range_band / m.to_f})
#     .height(y.range_band / m.to_f)
#     .left(0)
#     .width(x)
#     .fillStyle(lambda { colors.scale(self.index)})

# # #/* The value label. */
# # bar.anchor("right").add(pv.Label)
# #     .textStyle("white")
# #     .text(lambda {|d| "%0.1f" % d});

# #/* The variable label. */
# # bar.parent.anchor("left").add(pv.Label)
# #     .textAlign("right")
# #     .textMargin(5)
# #     .text(lambda {"ABCDEFGHIJK"[self.parent.index,1]});

# #/* X-axis ticks. */
# vis.add(pv.Rule)
#     .data(x.ticks(5))
#     .left(x)
#     .strokeStyle(lambda {|d| d!=0 ? "rgba(255,255,255,.3)" : "#000"})
#   .add(pv.Rule)
#     .bottom(0)
#     .height(5)
#     .strokeStyle("#000")
#   .anchor("bottom").add(pv.Label)
#     .text(x.tick_format);

# vis.render();
# vis

#   p.label(:anchor=>'top')
#     .fontSize(24)
#     .text("Users Editing per 4 hour periods")
# end
# vis

Error: there was a problem executing system command: visible(lambda { (self.index % 3)==0})


"test"

testing: {{x}}